In [14]:
# Imports for plots and system
import pandas as pd
import pickle

C:\Users\agraw\AppData\Local\Temp\ipykernel_8628\914992883.py:9: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [15]:
# Imports for ML models
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier, XGBRFClassifier
import catboost as cb

C:\Users\agraw\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [16]:
# Imports for Training and accuracy
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [21]:
# Load dataset into dataframe
df = pd.read_csv('feature_extracted_dataset.csv')
df.head()

,mfcc1_mean,mfcc1_var,mfcc2_mean,mfcc2_var,mfcc3_mean,mfcc3_var,mfcc4_mean,mfcc4_var,mfcc5_mean,mfcc5_var,...,spectral_bandwidth_mean,spectral_bandwidth_var,spectral_rolloff_mean,spectral_rolloff_var,zero_crossing_rate_mean,zero_crossing_rate_var,harmony_mean,harmony_var,tempo,label
0,-119.018040,2494.4453,125.080345,262.11307,-23.394335,365.39410,41.321730,181.91304,-5.931897,152.47517,...,1972.334258,117272.64020,3714.063439,1.083179e+06,0.081851,0.000558,-0.000047,0.008388,129.199219,blues
1,-125.977150,2046.7885,122.374200,218.45811,-20.679749,233.20865,50.035050,143.88158,-11.250690,139.74892,...,2010.751494,65940.66624,3870.510442,6.721332e+05,0.087173,0.001030,-0.000098,0.004937,123.046875,blues
2,-132.618150,3794.8044,114.815520,246.77928,-14.718568,192.96564,50.017944,153.58598,-0.424878,122.80047,...,2088.184750,73391.49800,4000.206581,7.859502e+05,0.071383,0.000425,-0.000007,0.012531,123.046875,blues
3,-118.325325,2505.8510,131.587170,323.43292,-18.493637,109.73642,39.478863,195.06105,-12.992122,140.52765,...,1967.920582,79805.90135,3579.149639,9.032748e+05,0.069426,0.000304,-0.000036,0.008459,123.046875,blues
4,-106.182110,2124.5708,134.275960,209.58644,-19.761967,173.80823,39.920500,111.32418,-14.074066,99.91732,...,1954.633566,57359.69560,3480.096905,5.889230e+05,0.070102,0.000288,-0.000016,0.009666,123.046875,blues


In [22]:
# Converting the label values to a range of 0 to 9
label_encoder = preprocessing.LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

# Seperating the features and labels
X = df.drop(['label'],axis=1)
y = df['label']

0       0
1       0
2       0
3       0
4       0
       ..
9985    9
9986    9
9987    9
9988    9
9989    9
Name: label, Length: 9990, dtype: int32

In [23]:
# Normalizing the feature values from 0 to 1
min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(X)

# Saving the Normalizing model for prediction use later
with open(f'Scalers/minmax.pickle', 'wb') as handle:
    pickle.dump(min_max_scaler, handle, protocol=pickle.HIGHEST_PROTOCOL)

# New data frame with the new scaled data. 
cols = X.columns
X = pd.DataFrame(np_scaled, columns = cols)

In [41]:
# Splitting the dataset into training and testing data 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=111)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((6993, 55), (2997, 55), (6993,), (2997,))

In [42]:
# Function to train and assess model
def model_assess(model, title = "Default"):
    model.fit(X_train, y_train)
    preds = model.predict(X_test)

    # Dump model into Binary pickle file for prediction use later
    with open(f'Models/{title}.pickle', 'wb') as handle:
        pickle.dump(model, handle, protocol=pickle.HIGHEST_PROTOCOL)

    # Printing the accuracy
    print('Accuracy', title, ':', round(accuracy_score(y_test, preds), 5), '\n')


In [43]:
# Naive Bayes
nb = GaussianNB()
model_assess(nb, "NaiveBayes")

Accuracy NaiveBayes : 0.52219 



In [44]:
# Stochastic Gradient Descent
sgd = SGDClassifier(max_iter=5000, random_state=0)
model_assess(sgd, "StochasticGradientDescent")

Accuracy StochasticGradientDescent : 0.66567 



In [45]:
# KNN
knn = KNeighborsClassifier(n_neighbors=19)
model_assess(knn, "KNN")

Accuracy KNN : 0.81481 



In [46]:
# Decission trees
tree = DecisionTreeClassifier()
model_assess(tree, "DecissionTrees")

Accuracy DecissionTrees : 0.63197 



In [47]:
# Random Forest
rforest = RandomForestClassifier(n_estimators=1000, max_depth=10, random_state=0)
model_assess(rforest, "RandomForest")

Accuracy RandomForest : 0.79813 



In [48]:
# Support Vector Machine
svm = SVC(decision_function_shape="ovo")
model_assess(svm, "SupportVectorMachine")

Accuracy SupportVectorMachine : 0.74241 



In [49]:
# Logistic Regression
lg = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial')
model_assess(lg, "LogisticRegression")

Accuracy LogisticRegression : 0.67968 



C:\Users\agraw\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [50]:
# Neural Nets
nn = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5000, 10), random_state=1)
model_assess(nn, "NeuralNets")

C:\Users\agraw\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\neural_network\_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Accuracy NeuralNets : 0.68569 



In [51]:
# catboost
cbc = cb.CatBoostClassifier(verbose=0, eval_metric='Accuracy', loss_function='MultiClass')
model_assess(cbc,"CatBoostClassifier")

Accuracy CatBoostClassifier : 0.89523 



In [52]:
# Cross Gradient Booster
xgb = XGBClassifier(n_estimators=1000, learning_rate=0.05)
model_assess(xgb, "CrossGradientBooster")

C:\Users\agraw\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\agraw\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[01:14:13] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy CrossGradientBooster : 0.8969 



In [53]:
# Cross Gradient Booster (Random Forest)
xgbrf = XGBRFClassifier(objective= 'multi:softmax')
model_assess(xgbrf, "CrossGradientBoosterRandomForest")

C:\Users\agraw\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\agraw\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[01:22:21] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy CrossGradientBoosterRandomForest : 0.72706 

